## Sources:
https://github.com/maxbozza/HOG-SVMlight-trainer/blob/master/train_SVMlight.py

https://github.com/aarcosg/object-detector-svm-hog-python


2-) 

* İlk ödevde ‘images/jpg/building.tif’ görüntüsü ile geliştirdiğiniz ölçek uzayı üzerinde 
yerel maksimum/yerel minimum noktaları belirlenecektir. 

* Ölçek uzayı olarak DoG filtreleme neticesinde ürettiğiniz 7 görüntüyü kullanabilirsiniz. 

* Yerel maksimum noktaları her bir pikselin 26-komşuluğu (9 komşu bir önceki ölçek, 8 komşu mevcut ölçek, 9 komşu bir sonraki ölçek için) kontrol edilerek belirlenecektir. 

* Yerel minimum için ise yerel maksimum bulmada kullandığınız sürecin aynısını ölçek uzayının işaretini tersine çevirerek çalıştırınız. 

* Elde ettiğiniz yerel maksimum ve minimum noktalarının tam sayı değerleri üzerinde uygun bir eşik değeri kullanınız (0,005 gibi küçük bir değer olabilir veya noktaların tamsayı değerlerine göre belirlenmiş bir değer olabilir, örneğin 0,05 * en büyük tamsayı değeri gibi). 

* Son olarak belirlediğiniz yerel noktaların kenar olma olasılığı yüksek olanlarını temizlemek için Hessian matrisine bağlı aşağıdaki koşulu kullanınız. 

* Elde ettiğiniz ilgi noktalarını ‘building.tif’ görüntüsü üzerinde genişletme ile gösteriniz. 

0 ≤ 𝑡𝑟(𝐻𝜎)2 \cfrac det(𝐻𝜎)≤12

𝑘𝑖 𝑏𝑢𝑟𝑎𝑑𝑎 𝐻𝜎 =[𝐿𝜎𝑥𝑥 𝐿𝜎𝑥𝑦 𝐿𝜎𝑦𝑥 𝐿𝜎𝑦𝑦] 𝑣𝑒 𝐿𝜎𝑥𝑥 =𝜕2(𝐿𝜎) / 𝜕𝑥2